<a href="https://colab.research.google.com/github/Heesung0706/Heesung.Hwnag/blob/main/MLL%EA%B3%BC%EC%A0%9C_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. XGBoost 설치

In [2]:
!pip install xgboost
!pip install pycaret[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 11.2 MB

#2. 파일 불러오기

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report


myfile = '/content/drive/MyDrive/FlowData.csv'

In [81]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier

#3. 데이터를 탐색 및 분석

In [57]:
df = pd.read_csv('/content/drive/MyDrive/FlowData.csv')

df.head()

,Category,sex,Skills,Challenges,Flow,Fun,Enjoyment_In,Enjoyment_Ex,Enjoyment_All
0,1,0,5,5,4.67,5,3.75,4.25,4.00
1,1,0,2,5,4.11,5,4.50,5.00,4.75
2,1,0,3,4,3.67,4,3.00,3.25,3.13
3,1,0,2,6,4.22,6,3.25,5.00,4.13
4,1,0,1,1,4.44,6,2.75,4.25,3.50


In [58]:
df.describe()

,Category,sex,Skills,Challenges,Flow,Fun,Enjoyment_In,Enjoyment_Ex,Enjoyment_All
count,813.000000,813.000000,813.000000,813.000000,813.000000,813.000000,813.000000,813.000000,813.000000
mean,1.482165,0.337023,3.667897,3.819188,4.091796,4.558426,4.362854,4.331181,4.392485
std,0.499989,0.472984,1.252747,1.162012,1.065574,1.122143,1.033064,1.008976,0.759972
min,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,0.000000,3.000000,3.000000,3.560000,4.000000,4.000000,4.000000,4.000000
50%,1.000000,0.000000,4.000000,4.000000,4.000000,5.000000,4.500000,4.250000,4.500000
75%,2.000000,1.000000,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.880000
max,2.000000,1.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 813 entries, 0 to 812
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Category       813 non-null    int64  
 1   sex            813 non-null    int64  
 2   Skills         813 non-null    int64  
 3   Challenges     813 non-null    int64  
 4   Flow           813 non-null    float64
 5   Fun            813 non-null    int64  
 6   Enjoyment_In   813 non-null    float64
 7   Enjoyment_Ex   813 non-null    float64
 8   Enjoyment_All  813 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 57.3 KB


#4. 데이터를 x (feature)와 y (target) 로 분리

In [60]:
x = df.drop('Flow', axis=1) # Features
y = df['Flow']              # Target

#5. Target 변수 Label Encoding

In [62]:
label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y)

#6. 학습용 데이터 분리

In [73]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#7. 알고리즘 XGBoost 선택 및 학습 모델 생성

In [74]:
xgb_classifier = xgb.XGBClassifier(random_state=42)

xgb_classifier.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

#8. 테스트 feature 데이터로 모델 예측

In [75]:
y_pred = xgb_classifier.predict(x_test)

#9. 모델 평가

In [96]:
report_xgb = classification_report(y_test, y_pred)

print(report_xgb)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           2       0.50      0.31      0.38        13
           3       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         0
           9       0.14      0.14      0.14        14
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00         1
          13       0.20      0.17      0.18         6
          14       0.00      0.00      0.00         1
          15       0.33      0.14      0.20         7
          16       0.00      0.00      0.00         5
          17       0.00      0.00      0.00         5
          18       0.46      0.56      0.51        34
          19       0.00      0.00      0.00         2
          20       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.